# 단일 입력 PromptTemplate

In [1]:
from langchain.prompts import PromptTemplate

In [2]:
import langchain
langchain.__version__

'0.3.25'

In [10]:
template_str=(
    '당신은 최고 수준의 마케팅 카피라이터입니다. \n'
    '아래 제품의 매력적인 홍보 문구를 200자 내외로 작성하십시오. \n\n'
    '제품명: {product_name}'
)
prompt = PromptTemplate.from_template(template_str)
formatted_prompt= prompt.format(product_name='초슈퍼카메라 x100')
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터입니다. 
아래 제품의 매력적인 홍보 문구를 200자 내외로 작성하십시오. 

제품명: 초슈퍼카메라 x100


In [11]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model='gpt-4.1-nano',temperature=0)
chain=prompt|llm
result=chain.invoke({'product_name':formatted_prompt})
print(result.content)

초슈퍼카메라 x100으로 순간을 놓치지 마세요! 선명한 화질과 빠른 반응속도로 어떤 순간도 생생하게 담아내는 최고의 선택. 프로처럼 찍는 즐거움, 지금 바로 경험하세요!


# 다중 입력 PromptTemplate

In [16]:
multi_template_str=(
    '아래 뉴스 기사 제목과 키워드입니다\n'
    '이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요\n\n'
    '제목: {title}\n'
    '키워드: {keywords}\n'
)

summary_prompt=PromptTemplate(template=multi_template_str, input_variables=['title','keywords'])
sample_title='인공지능 기술과 발전과 미래'
sample_keywords= '머신러닝, 딥러닝, 산업 혁신'
formatted_summary_prompt=summary_prompt.format(title=sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

아래 뉴스 기사 제목과 키워드입니다
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하세요

제목: 인공지능 기술과 발전과 미래
키워드: 머신러닝, 딥러닝, 산업 혁신



In [18]:
result_summary=(summary_prompt | llm).invoke({
    'title': sample_title,
    'keywords': sample_keywords
})
print(result_summary.content)

인공지능 기술은 머신러닝과 딥러닝의 발전을 바탕으로 산업 전반에 혁신을 일으키고 있으며, 앞으로도 지속적인 발전을 통해 다양한 분야에서 새로운 가능성을 열어갈 것으로 기대됩니다.
